# SSA Object for IVOA protocols

In [70]:
# imports
from astropy.io.votable.tree import VOTableFile, Resource, Field, Info, Param
from astropy.io.votable.tree import Table as voTable
from astropy.io.votable import from_table

from astropy.coordinates import SkyCoord

import specdb
from specdb.specdb import IgmSpec
from specdb import ssa as sssa

## Fiddling with QueryData and VOTables

### Load

In [5]:
tst_file = specdb.__path__[0]+'/tests/files/IGMspec_DB_v02_debug.hdf5'

In [11]:
igmsp = IgmSpec(dbfile=tst_file)

### Generate VOTable

In [73]:
reload(sssa)
ssa = sssa.SSAInterface(igmsp)

In [74]:
votable = ssa.querydata('0.0019,17.7737', SIZE=1e-3)

Your search yielded 1 match[es] within radius=0.001 deg


In [75]:
votable.to_xml('tmp.xml')

### Fiddle

In [35]:
coord = SkyCoord(ra=0.0019, dec=17.7737, unit='deg')
subcat = igmsp.qcat.cat_from_coords(coord)
subcat['RA'].unit = u.deg
subcat

Your search yielded 1 matches from 1 input coordinates


flag_group,sig_zem,flag_zem,RA,DEC,STYPE,zem,IGM_ID
,,,deg,,,,
int64,float64,str10,float64,float64,str3,float64,int64
1,-1.0,BOSS_PCA,0.00189828518376,17.7737391299,QSO,2.30763868197,0


In [77]:
len(votable.resources[0].tables)

1

In [53]:
#votable = from_table(subcat)
votable = VOTableFile()

In [55]:
resource = Resource(type="results")
votable.resources.append(resource)

In [56]:
status_info = Info(name='QUERY_STATUS', value='OK')

In [57]:
votable.resources[0].infos.append(status_info)

In [61]:
votable.resources[0].tables.append(voTable(subcat))

In [62]:
votable.to_xml('tmp.xml')

In [63]:
tbl = votable.resources[0].tables[0]

In [45]:
pub_param = Param(tbl, name="Publisher", utype="ssa:Curation.Publisher", ucd=" meta.curation", datatype="char", arraysize="*", value="JXP")

In [46]:
pub_param

<PARAM ID="Publisher" arraysize="*" datatype="char" name="Publisher" ucd=" meta.curation" utype="ssa:Curation.Publisher" value="JXP"/>

In [47]:
tbl.params.append(pub_param)

In [49]:
pub_param.description = "Here is a dummy desc"

In [51]:
votable.resources[0].description = "igmspec query"

In [64]:
voTable(subcat)

<Table length=0>
<No columns>

In [65]:
subcat

flag_group,sig_zem,flag_zem,RA,DEC,STYPE,zem,IGM_ID
,,,deg,,,,
int64,float64,str10,float64,float64,str3,float64,int64
1,-1.0,BOSS_PCA,0.00189828518376,17.7737391299,QSO,2.30763868197,0
